In [1]:
#Import packages and data
import pandas as pd
import datetime
import numpy as np
covid_patient_demographics = pd.read_csv('Data/covid/patient_demographics.csv', low_memory = False)
df = pd.read_csv('Data/Opioid_1/patient_demographics.csv', low_memory = False)
df1 = pd.read_csv('Data/Opioid_2/patient_demographics.csv', low_memory = False)
opioid_patient_demographics = pd.concat([df,df1], ignore_index=True)
patient_demo = pd.concat([covid_patient_demographics,opioid_patient_demographics], ignore_index=True)

In [8]:
#Dealing with all of the categorical variables, these will later bet one hot encoded
trim_demo = patient_demo.drop_duplicates()
trim_demo = trim_demo[['patient_num','sex_c','birth_date_shifted','marital_status','employment_status','race','ethnicity','language','state_name']]
trim_demo = trim_demo.drop_duplicates()
trim_demo['sex_c'] = trim_demo['sex_c'] -1 
trim_demo['age'] = (pd.to_datetime('2021-7-01') - pd.to_datetime(trim_demo['birth_date_shifted']))/np.timedelta64(1,'Y')
trim_demo['marital_status'] = trim_demo['marital_status'].map({'Married':'Married (Marital Status)', 'Single':'Single (Marital Status)', 'Divorced':'Divorced (Marital Status)',
                                                               'Widowed':'Widowed (Marital Status)', 'Unknown':'Unknown (Marital Status)','Legally Separated':'Legally Separated (Marital Status)',
                                                               'Significant Other':'Significant Other (Marital Status)', 'Patient Refused':'Patient Refused (Marital Status)'},na_action='ignore')
trim_demo['marital_status'].fillna(value = 'Patient refused (Marrital Status)', inplace = True)


trim_demo['employment_status'] = trim_demo['employment_status'].map({'Full Time':'Full Time (Employment)', 'Not Employed': 'Not Employed (Employment)',
                                                                     'Retired':'Retired (Employment)', 'Self Employed':'Self Employed (Employment)','Part Time':'Part Time (Employment)',
                                                                     'Student - Full Time':'Student - Full Time (Employment)', 'Disabled':'Disabled (Employment)', 'Unknown': 'Unknown (Employment)',
                                                                     'Student - Part Time': 'Student - Part Time (Employment)', 'Patient Refused':'Patient Refused (Employment)',
                                                                     'On Active Military Duty':'On Active Military Duty (Employment)'},na_action='ignore')
trim_demo['employment_status'].fillna(value = 'Unknown (Employment)', inplace = True)

trim_demo['race'].fillna(value = 'Unknown (Race)', inplace = True)
trim_demo['race'].replace({"Unknown": 'Unknown (Race)'}, inplace=True)

trim_demo['ethnicity'].fillna(value = 'Unknown (Ethnicity)', inplace = True)
trim_demo['ethnicity'].replace({"Unknown": 'Unknown (Ethnicity)'}, inplace=True)

trim_demo['language'].fillna(value = 'Need to Obtain (Language)', inplace = True)
trim_demo['language'].replace({"Need to Obtain": 'Need to Obtain (Language)'}, inplace=True)
trim_demo['state_name'].fillna(value = 'Unknown state', inplace = True)

In [13]:
#Define the columns of our pivot table and then generate the pivote table
col = np.concatenate([['sex_c'],trim_demo.marital_status.unique(),trim_demo.employment_status.unique(),
                      trim_demo.race.unique(),trim_demo.ethnicity.unique(),trim_demo.language.unique(),trim_demo.state_name.unique(),['age']])
demographic_df = pd.DataFrame(index=trim_demo.patient_num.unique(), columns=col)

In [23]:
#Fill the pivot table then save it as a csv
for index, row in trim_demo.iterrows():
  demographic_df.loc[row['patient_num'],'sex_c'] = row['sex_c']
  demographic_df.loc[row['patient_num'],row['marital_status']] = 1
  demographic_df.loc[row['patient_num'],row['employment_status']] = 1
  demographic_df.loc[row['patient_num'],row['race']] = 1
  demographic_df.loc[row['patient_num'],row['ethnicity']] = 1
  demographic_df.loc[row['patient_num'],row['language']] = 1
  demographic_df.loc[row['patient_num'],row['state_name']] = 1
  demographic_df.loc[row['patient_num'],'age'] = row['age']
demographic_df.fillna(0, inplace = True)
demographic_df.to_csv('Data/data_set/demo_df.csv')

In [ ]:
#Delete our old data to conserve space then load new data
del patient_demo  
c_med_orders = pd.read_csv('Data/covid/medication_orders.csv', low_memory = False)
df = pd.read_csv('Data/Opioid_1/medication_orders.csv', usecols=['patient_num','pharm_class'], low_memory = False)
df1 = pd.read_csv('Data/Opioid_2/medication_orders.csv', usecols=['patient_num','pharm_class'], low_memory = False)
o_med_orders = pd.concat([df,df1], ignore_index=True)
med_orders = pd.concat([c_med_orders,o_med_orders], ignore_index=True)
med_admin = pd.read_csv('Data/covid/medications_administered.csv', usecols=['patient_num','pharm_class'],low_memory = False)

In [ ]:
#Generate our columns and index for the pivot table
col2 = np.concatenate([med_orders.pharm_class.unique(),med_admin.pharm_class.unique()])
col2 = np.unique(col2.astype(str))
col2 = col2[~pd.isnull(col2)]
new_ind = np.unique(np.concatenate([med_orders.patient_num.unique(),med_admin.patient_num.unique()]))
med_ord_df = pd.DataFrame(index= med_orders.patient_num.unique(),columns = col2)

In [ ]:
#Fill the pivot table and save it
for index, row in med_orders.iterrows():
    med_ord_df.loc[row['patient_num'],row['pharm_class']] = 1
for index, row in med_admin.iterrows():
    med_ord_df.loc[row['patient_num'],row['pharm_class']] = 1
med_ord_df = med_ord_df.fillna(0)
med_ord_df.to_csv('Data/data_set/med_df.csv')